In [56]:
import numpy as np
import torch
import os

dataset = "SlateTopKBoredInfv0numitem100slatesize10_oracle_epsilon0.5_seed2023_n_users10"
data = torch.load(os.path.join("data", "datasets", f"{dataset}.pt"))
item_embeddings = torch.load(os.path.join("data", "datasets", "embeddings", f"{dataset}.pt"))

In [64]:
# make dataset where each row is a user containing a dict with the keys "slate" and "clicks"
data = torch.load(os.path.join("data", "datasets", f"{dataset}.pt")).observations

# now there are no users defined, but they can be inferred as each 100 interactions is a user
# so we can create a list of users where each user is a dict with the keys "slate" and "clicks"
users = []
user = dict(slate=[], clicks=[])
for slate, clicks in zip(data["slate"], data["clicks"]):
    user["slate"].append(slate)
    user["clicks"].append(clicks)
    if len(user["slate"]) == 100:
        users.append(user)
        user = dict(slate=[], clicks=[])
users

10

In [20]:
data.observations["slate"]

clicked_items_per_user = []

user_list = []
clicked_items = []

for i in range(len(data.observations["slate"])):
    slate =data.observations["slate"][i][0]
    clicks = data.observations["clicks"][i][0]
    # if the item was clicked, add it to the list
    for j in range(len(slate)):
        if clicks[j] == 1:
            clicked_items.append(int(slate[j]))
    if (i + 1) % 100 == 0:
        clicked_items_per_user.append(clicked_items)
        clicked_items = [] 


# create a user matrix where the rows are the users and the columns are the items. The clicked items are marked as the number of times a user has clicked on them
user_matrix = torch.zeros(len(clicked_items_per_user), 100)
for i in range(len(clicked_items_per_user)):
    for j in range(len(clicked_items_per_user[i])):
        user_matrix[i][clicked_items_per_user[i][j]] += 1
        
user_matrix

tensor([[ 0.,  0.,  0.,  0.,  0., 25.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          5.,  0.,  0.,  0.,  7.,  0.,  0., 16.,  0., 14.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          1.,  0.,  0.,  2.,  0.,  0.,  0.,  6.,  0., 15.,  1.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 19.,  0., 17.,  0.,  0.,  0.,
          5.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  6.,  0.,  0.,  0.,  0.,  0., 13.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.

In [31]:
"""Code based on https://github.com/naver/gems/blob/277a85fe971fdc736e4b292452b895631b4a1224/GeMS/modules/MatrixFactorization/"""

import torch
import torch.nn as nn
import torch.optim as optim

# Assuming user_matrix and item_embeddings are already defined tensors
# For example:
# user_matrix = torch.randn(10, 100)        # 10 users, 100 items
# item_embeddings = torch.randn(100, 10)    # 100 items, embedding dimension 10

print(user_matrix.shape)       # Should be torch.Size([10, 100])
print(item_embeddings.shape)   # Should be torch.Size([100, 10])

def sample_items(num_items, shape):
    """
    Randomly sample a number of items.
    Parameters
    ----------
    num_items: int
        Total number of items from which we should sample:
        the maximum value of a sampled item id will be smaller
        than this.
    shape: int or tuple of ints
        Shape of the sampled array.
    Returns
    -------
    items: np.array of shape [shape]
        Sampled item ids.
    """

    res_items = np.random.randint(0, num_items, shape, dtype=np.int64)
    return res_items

class DotProdScorer(nn.Module):
    def __init__(self, device):
        super(DotProdScorer, self).__init__()

        # Define the device_ops
        self.device = device

    def forward(self, user_embeddings, item_embeddings):
        # Scores based on the learned user/item embeddings
        if self.training:
            assert user_embeddings.size()[0] == item_embeddings.size()[0] # Equals to batch_size
            # Score user-item pairs aligned in user_embeddings and item_embeddings
            scores = (user_embeddings * item_embeddings).sum(-1).squeeze()
            ## Shape of scores: (batch_size)
        else:
            # Score every pair made of a row from user_embeddings and a row from item_embeddings
            scores = torch.mm(user_embeddings, item_embeddings.t())
            ## Shape of scores: (batch_size, num_item)

        return scores

def bpr_loss(positive_score, negative_score):
    """
    Bayesian Personalised Ranking loss
    Args:
        positive_score: (tensor<float>) predicted scores for known positive items
        negative_score: (tensor<float>) predicted scores for negative sample items
    Returns:
        loss: (float) the mean value of the summed loss
    """
    eps = 1e-7 # Smooth the argument of the log to prevent potential numerical underflows
    loss = -torch.log(torch.sigmoid(positive_score - negative_score) + eps)
    return loss.mean()

class MatrixFactorization(nn.Module):
    """
        Implementation of the matrix factorization with a BPR loss and trained with SGD
    """
    def __init__(self, num_user, num_item, device_embed, device_ops, embedd_dim=10, lr_embedd=0.0001, num_neg_sample=1, weight_decay=0.0, train_val_split=0.1):
        super(MatrixFactorization, self).__init__()

        self.num_user = num_user
        self.num_item = num_item
        self.embed_dim = embedd_dim
        self.lr = lr_embedd
        self.num_neg_sample = num_neg_sample
        self.device_embed = device_embed
        self.device_ops = device_ops
        self.weight_decay = weight_decay
        # Embeddings
        self.user_embeddings = nn.Embedding(num_user, self.embed_dim) # User embeddings to be learned
        self.item_embeddings = nn.Embedding(num_item, self.embed_dim) # Item embeddings to be learned
        nn.init.xavier_uniform_(self.user_embeddings.weight, gain=1)
        nn.init.xavier_uniform_(self.item_embeddings.weight, gain=1)
        self.user_embeddings = self.user_embeddings.to(device_embed)
        self.item_embeddings = self.item_embeddings.to(device_embed)

        # Components of the model
        self.scorer = DotProdScorer(device_ops).to(device_ops)

        # Optimizer
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

    def predict(self, user_ids, item_ids=None):
        """
        Compute the score predictions at test time
        Args:
            user_ids: (array<int>) users for whom to recommend items
            item_ids: (array<int>) items for which prediction scores are desired; if not provided, predictions for all
            items will be computed
        Returns:
            scores: (tensor<float>) predicted scores for all items in item_ids
        """
        batch_user_embeddings = self.user_embeddings(user_ids).to(self.device_ops)
        ## Shape of batch_user_embeddings: (batch_size, embed_dim)

        if item_ids is None:
            item_ids = np.arange(self.num_item)
        item_ids = torch.tensor(item_ids, dtype=torch.long, device=self.device_embed)
        batch_item_embeddings = self.item_embeddings(item_ids).to(self.device_ops)
        ## Shape of batch_item_embeddings: (num_item, embed_dim)

        scores = self.scorer(batch_user_embeddings, batch_item_embeddings)
        ## Shape of scores: (batch_size, num_item)
        return scores

    def forward(self, batch):
        # Unpack the content of the minibatch
        user_ids = batch['user_ids']
        item_ids = batch['item_ids']

        # Fetch the user embeddings for the minibatch
        batch_user_embeddings = self.user_embeddings(user_ids).to(self.device_ops)
        ## Shape of batch_user_embeddings: (batch_size, embed_dim)

        # Fetch the (positive) item embeddings for the minibatch
        batch_item_embeddings = self.item_embeddings(item_ids).to(self.device_ops)
        ## Shape of batch_item_embeddings: (batch_size, embed_dim)

        # Calculate the recommendation loss on the minibatch using BPR
        positive_score = self.scorer(batch_user_embeddings, batch_item_embeddings)

        ## Shape of positive_score: (batch_size)
        loss = torch.tensor(0.0, dtype=torch.float, device=self.device_ops)
        for i in range(self.num_neg_sample):
            # Negative sampling
            negative_item_ids = sample_items(self.num_item, item_ids.size())
            negative_item_ids = torch.tensor(negative_item_ids, dtype=torch.long, device=self.device_embed)
            batch_negative_item_embeddings = self.item_embeddings(negative_item_ids).to(self.device_ops)
            ## Shape of batch_negative_item_embeddings: (batch_size, embed_dim)
            negative_score = self.scorer(batch_user_embeddings, batch_negative_item_embeddings)
            ## Shape of negative_score: (batch_size)
            # Compute the BPR loss on the positive and negative scores while masking padded elements in the sequences
            loss += bpr_loss(positive_score, negative_score)


        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss

# Correctly assign number of items
n_users = user_matrix.size(0)          # 10
n_items = item_embeddings.shape[0]     # Corrected: use shape to get the first dimension
embedding_dim = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MatrixFactorization(n_users, n_items, device, device)

# Define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Prepare training data: all possible user-item pairs
# Number of pairs: n_users * n_items
user_indices = torch.LongTensor([u for u in range(n_users) for _ in range(n_items)])  # Shape: [1000]
item_indices = torch.LongTensor([i for i in range(n_items) for _ in range(n_users)])  # Shape: [1000]
ratings = user_matrix.view(-1)   
                                            # Shape: [1000]

# Optionally, move data to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
user_indices = user_indices.to(device)
item_indices = item_indices.to(device)
ratings = ratings.to(device)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    predictions = model(n_users, n_items)  # Shape: [1000]
    
    # Compute loss
    loss = criterion(predictions, ratings)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")

# Retrieve the trained user embeddings
user_embeddings = model.user_embedding.weight.data
print(user_embeddings)



torch.Size([10, 100])
(100, 10)


TypeError: MatrixFactorization.forward() takes 2 positional arguments but 3 were given

In [5]:
user_matrix.shape

torch.Size([100000, 100])

In [45]:
item_embeddings.shape

(100, 10)

In [12]:
item_embeddings

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.3152781 , 0.        , 0.        , 0.94899932],
       [0.        , 0.40346547, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.649015  , 0.64497685],
       [0.        , 0.41663766, 0.69163858, 0.        , 0.        ,
        0.        , 0.        , 0.58995689, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.33808886, 0.        , 0.        , 0.94111419],
       [0.95641525, 0.        , 0.        , 0.        , 0.29201005,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.03330959, 0.        , 0.8047671 , 0.59265537],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.86394508, 0.50358605, 0.        , 0.        , 0.        ],
       [0.1750753 , 0.84795926, 0.       

In [16]:
print(model.item_embedding.weight[0])
print(model.item_embedding.weight[3])
print(model.item_embedding.weight[5])


tensor([0.4938, 0.2779, 0.3481, 0.6168, 0.3508], grad_fn=<SelectBackward0>)
tensor([0.6394, 0.4799, 0.2776, 0.6288, 0.4236], grad_fn=<SelectBackward0>)
tensor([0.5125, 0.2788, 0.4804, 0.3227, 0.2684], grad_fn=<SelectBackward0>)


In [47]:
ratings

tensor([0., 0., 0.,  ..., 0., 9., 0.])

In [32]:
np.random.randint(0, 10, 5)

array([5, 7, 0, 3, 3], dtype=int32)

In [37]:
# get random elements from item_embeddings
samples = item_embeddings[np.random.randint(0, item_embeddings.shape[0], 5)]
samples

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.84526873, 0.09005205, 0.        , 0.52669858],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.84526873, 0.09005205, 0.        , 0.52669858],
       [0.41056857, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.91182973, 0.        ],
       [0.        , 0.        , 0.70198059, 0.        , 0.        ,
        0.        , 0.        , 0.1866325 , 0.68730747, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.5942942 , 0.55739437, 0.57976367, 0.        ]])

In [36]:
def sample_items(num_items, shape):
    """
    Randomly sample a number of items.
    Parameters
    ----------
    num_items: int
        Total number of items from which we should sample:
        the maximum value of a sampled item id will be smaller
        than this.
    shape: int or tuple of ints
        Shape of the sampled array.
    Returns
    -------
    items: np.array of shape [shape]
        Sampled item ids.
    """

    res_items = np.random.randint(0, num_items, shape, dtype=np.int64)
    return res_items

100

In [40]:
torch.tensor([b[0] for b in user_matrix], dtype = torch.long,)

TypeError: only integer tensors of a single element can be converted to an index

In [51]:
data = user_matrix
# assign each user in the data a unique id
data = [(i, data[i]) for i in range(len(data))]
[torch.nonzero(b[1], as_tuple=True)[0] for b in data]

[tensor([ 5, 14, 18, 21, 23, 39, 56, 59, 63, 65, 66, 83]),
 tensor([10, 22, 24, 28, 34, 58, 64, 89, 91, 95]),
 tensor([ 4, 24, 35, 45, 58, 59, 60, 72, 84]),
 tensor([ 5, 11, 21, 23, 39, 65, 70, 82]),
 tensor([30, 40, 53, 82, 96]),
 tensor([ 4, 35, 45, 58, 60, 62, 72, 78]),
 tensor([ 6, 17, 22, 42, 46, 53, 64, 80, 92]),
 tensor([ 1, 13, 21, 23, 30, 39, 52, 53, 57, 65, 66, 77, 96]),
 tensor([ 1, 13, 30, 41, 53, 57, 77, 80, 96]),
 tensor([ 3, 11, 23, 30, 40, 50, 65, 70, 82])]

In [54]:
[b[1].tolist() for b in data]

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  25.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  5.0,
  0.0,
  0.0,
  0.0,
  7.0,
  0.0,
  0.0,
  16.0,
  0.0,
  14.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  10.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  6.0,
  0.0,
  15.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  8.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  7.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  19.0,
  0.0,
  17.0,
  0.0,
  0.0,
  0.0,
  5.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0